In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import os
from PIL import Image
import io
import hashlib
import requests

In [16]:
from requests.structures import CaseInsensitiveDict

headers = CaseInsensitiveDict()
headers["Connection"] = "keep-alive"
# headers["Keep-Alive"] = "timeout=5, max=100"


In [17]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = []
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.append(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [18]:
def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url, headers=headers).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")


In [33]:
if __name__ == '__main__':
    s = Service('C:/Users/Suni/Desktop/chromedriver.exe')
    options = webdriver.ChromeOptions()
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-extensions")
    options.add_argument("--proxy-server='direct://'")
    options.add_argument("--proxy-bypass-list=*")
    options.add_argument("--start-maximized")
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--no-sandbox')
    options.add_argument('--ignore-certificate-errors')
    options.add_argument("--enable-webgl-developer-extensions")
    options.add_argument("--enable-webgl-draft-extensions")
    wd = webdriver.Chrome(service=s, options=options)
    wd.implicitly_wait(10)

    queries = ["immigration border"] 

    for query in queries:
        wd.get('https://google.com')
        search_box = wd.find_element(By.CSS_SELECTOR, 'input.gLFyf') #input box selector
        search_box.send_keys(query)
        links = fetch_image_urls(query, 550, wd)
        images_path = 'C:/Users/Suni/Desktop/images'
        
        for i in links:
            persist_image(images_path,query,i)            
    wd.quit()

C:\Users\Suni\AppData\Local\Temp/ipykernel_40380/647851267.py:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 200 search results. Extracting links from 0:200


C:\Users\Suni\AppData\Local\Temp/ipykernel_40380/647851267.py:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 552 image links, done!
SUCCESS - saved https://media.wired.com/photos/5d4e685315b58b000896c36f/master/pass/Business-Immigration-1150320306.jpg - as C:/Users/Suni/Desktop/images\immigration border\1b42ebb79c.jpg
SUCCESS - saved https://i.guim.co.uk/img/media/f216e151970c2565266c9c93f77389cc8c7e342a/1479_185_3541_2125/master/3541.jpg?width=1200&quality=85&auto=format&fit=max&s=6348d3eab9c0f0cfe5e28a8d74497c0c - as C:/Users/Suni/Desktop/images\immigration border\e80e356e7a.jpg
SUCCESS - saved https://static.dw.com/image/46448068_303.jpg - as C:/Users/Suni/Desktop/images\immigration border\418033c8ef.jpg
SUCCESS - saved https://www.aljazeera.com/wp-content/uploads/2021/10/2021-10-18T190004Z_1110206911_RC2ECQ9P0GXP_RTRMADP_3_USA-IMMIGRATION-MEXICO.jpg?resize=1200%2C630 - as C:/Users/Suni/Desktop/images\immigration border\870ae4de86.jpg
SUCCESS - saved https://media2.s-nbcnews.com/i/MSNBC/Components/Video/201804/nn_gsc_migrant_caravan_on_us_border_180429_1920x1080.jpg - as C:/Users/Su

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRLPgDczBULzj7s8eWds5U44qw91WKMqkhCRg&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\daab71dac8.jpg
SUCCESS - saved https://media.newyorker.com/photos/609197d3dab5a5b9b7487e8e/master/pass/Taldrid-Border.jpg - as C:/Users/Suni/Desktop/images\immigration border\d6c777ec76.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTLrEdi-_bXgEaOrcvTCO4TTurfLjg1mewsRg&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\eb582da8a2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRmEJN7HqwJW20oc5s0iAj6vvbRNnaxkZCOpQ&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\c0e6fb2426.jpg
ERROR - Could not save https://upload.wikimedia.org/wikipedia/commons/6/6d/2019_US_Mexico_Border_Crossing_apprehension_%2848036606282%29.jpg - cannot identify image file <_io.BytesIO object at 0x000001DBADB663E0>
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=

...

SUCCESS - saved https://immigrationimpact.com/wp-content/uploads/2021/12/immigration-impact-biden-border-ambitions.jpg - as C:/Users/Suni/Desktop/images\immigration border\3a68e13b3d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQRFMxgWvWPiZTesYGdphR6I5nQhszhFN74A&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\8f4c9b62e2.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTS2u3D4XJCNmmXjiVPTNtxuWPIE8S_3mZzuA&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\40c10c25ed.jpg
SUCCESS - saved https://www.voiceofsandiego.org/wp-content/uploads/2017/08/BorderTour_AH_2-e1503345419327-800x534.jpg - as C:/Users/Suni/Desktop/images\immigration border\77b0336fd3.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRWEMgoSvwiPPHD_d1plmqK7oNZeN3POc1qkw&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\e53f6aa74c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTQRFM

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ4cwvXHi4SCnVMMaeoR4tJVV9y21IEyFgSXQ&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\ed390368e0.jpg
SUCCESS - saved https://www.gannett-cdn.com/presto/2019/06/25/USAT/b1eac302-0701-431e-a7da-d3face6c78b9-Immigration_Texas.JPG?width=660&height=539&fit=crop&format=pjpg&auto=webp - as C:/Users/Suni/Desktop/images\immigration border\90fa8b7d91.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSgQGoHtGnBOrRh2GrxOK6YOtmw7FIPlXq6vg&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\fcb83e6338.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRHZZWtIe8iB-Zh41n06ApjwjnpMeXqJLrmug&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\6ce3039e26.jpg
SUCCESS - saved https://news.harvard.edu/wp-content/uploads/2021/05/051321_Border_Crisis_05-1200x800.jpg - as C:/Users/Suni/Desktop/images\immigration border\337f3fc3e8.jpg
SUCCESS - saved https://encry

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDH6CObP2TQkWTPpa0xIwdS096_PtguEidrQ&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\04600043e6.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRFmC3h2iHBef_yC_FK6xIB-OYL_Hg6FFp_EQ&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\af79cd052d.jpg
SUCCESS - saved https://wehco.media.clients.ellingtoncms.com/img/photos/2021/02/06/AP21018742609816_t800.jpg?90232451fbcadccc64a17de7521d859a8f88077d - as C:/Users/Suni/Desktop/images\immigration border\d95b5aef66.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRN1zXVg1lPAIfX9ZjgI2eAfNAZ6M4jm5DfnA&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\02d6e8badb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDH6CObP2TQkWTPpa0xIwdS096_PtguEidrQ&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\04600043e6.jpg
SUCCESS - saved https://images.wsj.net/im-4627

SUCCESS - saved https://static01.nyt.com/images/2021/03/15/us/politics/15dc-immig/15dc-immig-articleLarge-v3.jpg?quality=75&auto=webp&disable=upscale - as C:/Users/Suni/Desktop/images\immigration border\de02fb636d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSEks2aRjDpoIyi1tpiVG1a71jxPoGNP3RWRQ&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\4ad91749cb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRd9IbNiE9t0_XpaaKK1UQHmSj5gNtpO0WR5w&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\32ea6745b2.jpg
SUCCESS - saved https://images.axios.com/LDgXHT1cJ-JP0WnWQUDy0oXOeDA=/0x46:3000x1734/1920x1080/2021/06/05/1622920417884.jpg - as C:/Users/Suni/Desktop/images\immigration border\acd16b51e1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSD5T1Y90ZYxOwFQTw5MFLv0_YMFpGKDT0HqA&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\17d8f4491d.jpg
SUCCESS - saved https://encrypted-tbn0

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQzd2i06DjbSO0JnyyYVK-DGLNsG8jiJVOtQ&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\171dab4ba1.jpg
SUCCESS - saved https://cdnph.upi.com/svc/sv/upi/3271544536020/2018/1/ec627feae9c8be95fc5197bcef252785/Agents-arrest-more-than-30-at-immigration-protest-on-US-Mexico-border.jpg - as C:/Users/Suni/Desktop/images\immigration border\32a81fc19c.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTVgta6pFgovUzKEpmWeIY0qI52we6GGmWX4g&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\c906e381e0.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ12J6ZeLyE2zVmCZJl-6GMf9ne92bJ3KLBSw&usqp=CAU - as C:/Users/Suni/Desktop/images\immigration border\39d4c633d1.jpg
SUCCESS - saved https://media2.sacurrent.com/sacurrent/imager/u/original/21470941/border_shutterstock_459602719.jpg - as C:/Users/Suni/Desktop/images\immigration border\f2525dfb58.jpg
SUCCESS - saved htt